In [ ]:
from processing import convert_to_mne
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use('Qt5Agg')

path = f"C:/Users/Gianni/Desktop/MARCO/UNI/Magistrale/TESI/User Study/experiments/user_0/test_BCI_RELAXED_0/session.csv"

rescale = 1e6
fs = 250
chs = ["Fz", "C3", "Cz", "C4", "Pz", "PO7", "Oz", "PO8"]
columns = ["Fz", "C3", "Cz", "C4", "Pz", "PO7", "Oz", "PO8", "AccX", "AccY", "AccZ", "GyroX", "GyroY", "GyroZ",
           "CNT", "BAT", "VALID", "DeltaTime", "Trigger"]

if 'recordings' in path:
    delimiter = ','
else:
    delimiter = '\t'

data = pd.read_csv(filepath_or_buffer=path, delimiter=delimiter)
data.columns = columns[:data.shape[1]-1] + [columns[-1]]

print(data.head())

minutes = len(data)/(60*fs)
seconds = (len(data)/fs) % 60 
print(f'Recording duration: {int(minutes)} minutes and {int(seconds)} seconds')

trigger = data.iloc[:, -1].to_numpy(dtype=np.float64)
print(np.unique(trigger))

eeg = data.iloc[:, 0:8].to_numpy(dtype=np.float64)

raw_data = convert_to_mne(eeg, trigger, rescale=rescale, fs=fs, chs=chs, recompute=False)

              Fz           C3             Cz             C4           Pz  \
0  190942.640625  679530.5000   73831.750000 -706664.187500  628775.6250   
1  105692.195312  617680.0000 -171306.187500 -750000.125000  708153.6250   
2  252876.109375  639062.6875 -158795.296875 -639705.062500  750000.0625   
3  310802.281250  676236.3125  104325.875000 -213538.390625  750000.0625   
4  354708.968750  749830.6875   88355.468750   86712.882812  750000.0625   

             PO7           Oz           PO8      AccX      AccY      AccZ  \
0  289109.375000  716170.3750  750000.06250 -0.126709  1.082275 -0.037598   
1  229804.203125  648077.9375  750000.06250 -0.124756  1.088623 -0.040283   
2  236569.890625  664866.6250  637545.43750 -0.122314  1.088867 -0.039795   
3  336406.375000  740145.3125  479214.28125 -0.120117  1.089844 -0.043213   
4  547492.750000  750000.0625  750000.06250 -0.117188  1.086914 -0.042725   

      GyroX      GyroY    GyroZ        CNT  BAT  VALID     DeltaTime  Trigger  


In [40]:
# Compute PSD
Pxx = raw_data.compute_psd(fmin=0, fmax=fs/2)
Pxx.plot()
plt.show()

Effective window size : 8.192 (s)


In [41]:
filtered = raw_data.copy() # the method filters the signal in-place, so this time I
                      # want to preserve the original signal and filter just a
                      # temporary copy of it

# remove power line noise
filtered.notch_filter(50) 
filtered.notch_filter(60) 
# Apply band-pass filtering
filtered.filter(1,30) 

pxx_filt = filtered.compute_psd(fmin=0, fmax=50)
pxx_filt.plot()
plt.show()

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband e

In [42]:
filtered.plot()
plt.show()

Channels marked as bad:
none


### Channels Interpolation
Previously marked bad channels can be interpolated

In [32]:
## Interpolate bad channels
interp = filtered.copy()

# Mark the bad channels in the raw data
print(interp.info['bads'])

interp.interpolate_bads()
interp.plot()
plt.show()

[]
Setting channel interpolation method to {'eeg': 'spline'}.


C:\Users\Gianni\AppData\Local\Temp\ipykernel_10712\2769900535.py:7: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  interp.interpolate_bads()


Channels marked as bad:
none


### Cleaning data using ASR
Full example available: https://github.com/DiGyt/asrpy/blob/main/example.ipynb

In [33]:
## take the first 30 seconds of the data
training_data = interp.copy()
training_data.crop(tmin=5, tmax=65)
training_data.plot()
plt.show()

Channels marked as bad:
none


In [35]:
import asrpy

asr = asrpy.ASR(sfreq=fs, cutoff=15)
asr.fit(training_data)
cleaned = asr.transform(interp)
cleaned.plot()
plt.show()

Channels marked as bad:
none
